# Interpretation and plotting

The sparse networks created using BINN can be interpreted using various post-hoc interpretation methods. We chose to utilize SHAP to explain which nodes are important for classifications made by the BINN.

Similar to in the [BINN example](../binn_example), we load some example data, and generate the network and the BINN.

In [2]:
from binn import Network, BINN
import pandas as pd

input_data = pd.read_csv("../data/test_qm.csv")
translation = pd.read_csv("../data/translation.tsv", sep="\t")
pathways = pd.read_csv("../data/pathways.tsv", sep="\t")

# Create the model
network = Network(
    input_data=input_data,
    pathways=pathways,
    mapping=translation,
)

binn = BINN(
    pathways=network,
    n_layers=4,
    dropout=0.2,
    validate=False,
    residual=True
)

We load some test data and train the network.

In [4]:
from notebooks.util_for_examples import fit_data_matrix_to_network_input, generate_data
import torch
from pytorch_lightning import Trainer

design_matrix = pd.read_csv('../data/design_matrix.tsv', sep="\t")
protein_matrix = pd.read_csv('../data/test_qm.csv')

protein_matrix = fit_data_matrix_to_network_input(
    protein_matrix, features=network.inputs)

X, y = generate_data(protein_matrix, design_matrix=design_matrix)

dataloader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(torch.Tensor(X), torch.LongTensor(y)),
                                            batch_size=8,
                                            num_workers=12,
                                            shuffle=True)
trainer = Trainer(max_epochs=5)
trainer.fit(binn, dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Missing logger folder: /home/erikh/BINN/BINN/docs/lightning_logs

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 365 K 
1 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
365 K     Trainable params
0         Non-trainable params
365 K     Total params
1.464     Total estimated model params size (MB)
The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 4: 100%|██████████| 25/25 [00:00<00:00, 47.75it/s, loss=0.639, v_num=0, train_loss=0.640, train_acc=0.680]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 25/25 [00:00<00:00, 43.39it/s, loss=0.639, v_num=0, train_loss=0.640, train_acc=0.680]


Generate the BINNExplainer object.

In [5]:
from binn import BINNExplainer

test_data = torch.Tensor(X)
background_data = torch.Tensor(X)

explainer = BINNExplainer(binn)

Explain the network using SHAP. The value column in the resulting dataframe contains the SHAP values. Note that each source and target entity is now superceded by the layer index. This is so that we can keep track of copies of nodes. 

In [6]:
importance_df = explainer.explain(test_data, background_data)
importance_df.head()

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


,source,target,value,type,source layer,target layer
0,A0M8Q6_0,R-HSA-166663_1,0.000525,0,0,1
1,A0M8Q6_0,R-HSA-166663_1,0.001632,1,0,1
2,A0M8Q6_0,R-HSA-977606_1,0.000525,0,0,1
3,A0M8Q6_0,R-HSA-977606_1,0.001632,1,0,1
4,A0M8Q6_0,R-HSA-2029481_1,0.000525,0,0,1
